In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'eyes-open-or-closed:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1232901%2F2057400%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240226%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240226T034456Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da41d6d8e266c4493ef11e13d84104eb50c48c582cfb6af89c571d4d3f9e439246fcbbe3bff253cb33da0ad2cde73f1bdd2091f8fad119094e37e5d875fc6cd7a7c588385b8e3c3d3e7e4ca031a884b2297944da803c8b4447aed1062a6321106a8b8a45054e9e39e2a20ff8217d68e27c3f238420e2bb27b31960b1886f672c66d1436d37ec24bb7e77220b794737324a419439a576e439111c36ee5816873f3e73d8ae013e0e943ad85623079dffc3245f08ff3d29ec89ea70135393ff638a5de31b268962e0c499f6b887861286585f0c763df92faf36c85b82a8cad40edce45e6785161edaf3ef89658284deb7b3ceacbe4896be6c1895f528adc6a9c9e9a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 23747803 bytes downloaded
Downloaded and uncompressed: eyes-open-or-closed
Data source import complete.


In [ ]:
import tensorflow as tf
train_dir ='../input/eyes-open-or-closed/dataset/train'
test_dir  ='../input/eyes-open-or-closed/dataset/test'

In [ ]:
width, height = 86, 86
training=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                          rotation_range=7,
                                                          horizontal_flip=True,
                                                          validation_split=0.05
                                                         ).flow_from_directory(train_dir,
                                                                               class_mode = 'categorical',
                                                                               batch_size = 8,
                                                           target_size=(width,height),
                                                                              subset="training")
testing=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                         ).flow_from_directory(test_dir,
                                                                               class_mode = 'categorical',
                                                                               batch_size = 8,
                                                                               shuffle = False,
                                                           target_size=(width,height))
validing=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                          rotation_range=7,
                                                          horizontal_flip=True,
                                                         validation_split=0.05
                                                        ).flow_from_directory(train_dir,
                                                                              batch_size = 8,
                                                                              class_mode = 'categorical',
                                                           target_size=(width,height),subset='validation',shuffle=True)

Found 3230 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 170 images belonging to 2 classes.


In [ ]:
from keras.models import Sequential ,Model
from keras.layers import Dense ,Flatten ,Conv2D ,MaxPooling2D ,Dropout ,BatchNormalization  ,Activation ,GlobalMaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping ,ReduceLROnPlateau

In [ ]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.99)
EarlyStop = EarlyStopping(patience=10, restore_best_weights=True)
Reduce_LR = ReduceLROnPlateau(monitor='val_accuracy', verbose=2, factor=0.5, min_lr=0.00001)
callbacks = [EarlyStop, Reduce_LR]

In [ ]:
num_classes = 2
num_detectors=32

network = Sequential()

network.add(Conv2D(num_detectors, (3,3), activation='relu', padding = 'same', input_shape = (width, height, 3)))
network.add(BatchNormalization())
network.add(Conv2D(num_detectors, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Conv2D(2*num_detectors, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(Conv2D(2*num_detectors, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Conv2D(2*2*num_detectors, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(Conv2D(2*2*num_detectors, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Conv2D(2*2*2*num_detectors, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(Conv2D(2*2*2*num_detectors, (3,3), activation='relu', padding = 'same'))
network.add(BatchNormalization())
network.add(MaxPooling2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Flatten())

network.add(Dense(2 * num_detectors, activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.2))

network.add(Dense(2 * num_detectors, activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.2))

network.add(Dense(num_classes, activation='softmax'))

In [ ]:
network.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
network.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 86, 86, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 86, 86, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 86, 86, 32)        9248      
                                                                 
 batch_normalization_1 (Bat  (None, 86, 86, 32)        128       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 43, 43, 32)        0         
 D)                                                              
                                                        

In [ ]:
history = network.fit(training, validation_data=validing, epochs=20, callbacks=callbacks, verbose=2)

Epoch 1/20
404/404 - 218s - loss: 0.3181 - accuracy: 0.8808 - val_loss: 0.6692 - val_accuracy: 0.6706 - lr: 0.0010 - 218s/epoch - 539ms/step
Epoch 2/20
404/404 - 216s - loss: 0.1969 - accuracy: 0.9285 - val_loss: 0.3561 - val_accuracy: 0.8353 - lr: 0.0010 - 216s/epoch - 535ms/step
Epoch 3/20
404/404 - 213s - loss: 0.1089 - accuracy: 0.9628 - val_loss: 0.5716 - val_accuracy: 0.8059 - lr: 0.0010 - 213s/epoch - 528ms/step
Epoch 4/20
404/404 - 210s - loss: 0.0882 - accuracy: 0.9718 - val_loss: 0.0202 - val_accuracy: 0.9941 - lr: 0.0010 - 210s/epoch - 519ms/step
Epoch 5/20
404/404 - 214s - loss: 0.0570 - accuracy: 0.9820 - val_loss: 0.1243 - val_accuracy: 0.9588 - lr: 0.0010 - 214s/epoch - 530ms/step
Epoch 6/20


In [ ]:
val,los=network.evaluate(testing)

In [ ]:
import matplotlib.pyplot as plt

metrics = history.history
plt.plot(history.epoch, metrics['loss'])
plt.legend(['loss'])
plt.show()

In [ ]:
network.save('eyes.h5')